# Field Validation

In [4]:

import pandas as pd #reading dataset
import matplotlib as mpl    #plotting, visualization
import matplotlib.pyplot as plt
import tensorflow as tf #building model
from tensorflow.keras.models import Model
import numpy as np  #playing output with model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import sys
sys.path.append('../../')   # Add parent directory to Python path
from utils.Plots import *

In [ ]:
#load data
df_data = pd.read_csv('../../data/test/Accelerometer_ziyi.csv') 
df_data

,Date,NTP,GNSS-Time,Value
0,2025-03-28T16:00:59.302,2025-03-28 16:01:02.660,1743174062643,Curb Up
1,2025-03-28T16:01:38.562,2025-03-28 16:01:41.920,1743174101903,Curb Up
2,2025-03-28T16:01:49.559,2025-03-28 16:01:52.917,1743174112903,Curb Up
3,2025-03-28T16:01:56.627,2025-03-28 16:01:59.985,1743174119969,Curb Up
4,2025-03-28T16:02:06.915,2025-03-28 16:02:10.274,1743174130257,Curb Up
...,...,...,...,...
135,2025-03-28T16:22:22.830,2025-03-28 16:22:26.188,1743175346171,Curb Up
136,2025-03-28T16:22:28.231,2025-03-28 16:22:31.589,1743175351572,Curb Up
137,2025-03-28T16:22:30.179,2025-03-28 16:22:33.537,1743175353520,Curb Up
138,2025-03-28T16:22:42.238,2025-03-28 16:22:45.596,1743175365579,Curb Up


In [9]:
df_curb = pd.read_csv('../../data/test/ESP_curb.csv')
df_curb

,Date,NTP,GNSS-Time,Value
0,2025-03-28T16:00:59.302,2025-03-28 16:01:02.660,1743174062643,Curb Up
1,2025-03-28T16:01:38.562,2025-03-28 16:01:41.920,1743174101903,Curb Up
2,2025-03-28T16:01:49.559,2025-03-28 16:01:52.917,1743174112903,Curb Up
3,2025-03-28T16:01:56.627,2025-03-28 16:01:59.985,1743174119969,Curb Up
4,2025-03-28T16:02:06.915,2025-03-28 16:02:10.274,1743174130257,Curb Up
...,...,...,...,...
135,2025-03-28T16:22:22.830,2025-03-28 16:22:26.188,1743175346171,Curb Up
136,2025-03-28T16:22:28.231,2025-03-28 16:22:31.589,1743175351572,Curb Up
137,2025-03-28T16:22:30.179,2025-03-28 16:22:33.537,1743175353520,Curb Up
138,2025-03-28T16:22:42.238,2025-03-28 16:22:45.596,1743175365579,Curb Up


In [ ]:
#missing value


In [ ]:
# label curb, before and after 0.5s

In [ ]:
#segmentation

In [ ]:
#predit with 3 models
#plot the result with 3 models
#analyse